In [1]:
import pandas as pd
import wave
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import os 
import tensorflow as tf

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python

# This notebook creates the clips for the testing data, obtained from long 24 hours audios. 

Read in data frames

In [2]:
korup = pd.read_csv('/Users/rosameliacarioni/University/Thesis/code/data_frames/test/korup.csv')
pnnn_1 = pd.read_csv('/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_1.csv')
pnnn_2 = pd.read_csv('/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_2.csv')
pnnn_3 = pd.read_csv('/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_3.csv')
pnnn_4 = pd.read_csv('/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_4.csv')

In [ ]:
pnnn_1

# Transform audios to all have a sample rate of 8.000 

In [ ]:
# Time is passed in seconds
def change_one_file_rate (file):
    file_location = '/Volumes/Meli_disk/thesis_data/Sounds_ELP/' + file
    new_sample_rate = 8000
    try:
        audio = AudioSegment.from_wav(file_location)
        audio = audio.set_frame_rate(new_sample_rate)
        audio.export(file_location, format="wav")
    except FileNotFoundError:
        print ('ERROR ', file_location)
    
def change_all_files (files):
    for file in files: 
        change_one_file_rate(file)



Resample all files needed/used by the different data frames

In [ ]:
files_korup = korup['Begin File'].unique().tolist()
change_all_files(files_korup)

In [ ]:
files_pnnn_1 = pnnn_1['Begin File'].unique().tolist()
change_all_files(files_pnnn_1)

In [ ]:
files_pnnn_2 = pnnn_2['Begin File'].unique().tolist()
change_all_files(files_pnnn_2)

In [ ]:
files_pnnn_3 = pnnn_3['Begin File'].unique().tolist()
change_all_files(files_pnnn_3)

# Clip sounds containing gunshots

In [3]:
# Time is passed in seconds
def clip_audio_and_save (instance_of_df, extra_name):
    time_start = instance_of_df['File Offset (s)']
    time_duration = instance_of_df['delta time (s)']
    file_location = '/Volumes/Meli_disk/thesis_data/Sounds_ELP/' + instance_of_df['Begin File']
    file_name = '/Volumes/Meli_disk/thesis_data/test/Clips_ELP/gunshot/' + extra_name + str(instance_of_df.name) + '.wav'

    second_to_milliseconds = 1000

    try:
        audio = AudioSegment.from_wav(file_location)
    except FileNotFoundError:
        return None
    
    clip = audio[time_start*second_to_milliseconds: time_start*second_to_milliseconds + time_duration*second_to_milliseconds]
    clip.export(file_name, format='wav')
    return file_name

def clip_all_df (df, extra_name):
    files_not_found = set()
    df['Clip Location'] = df.apply(clip_audio_and_save, axis = 1, extra_name = extra_name)

    for index, row in df.iterrows():
        if row['Clip Location'] is None:
            files_not_found.add(row['Begin File'])
    return files_not_found

Call method to generate the clips from the 5 data frames 

In [4]:
korup_not_found = clip_all_df(korup, 'korup_')
print(korup_not_found)


set()


In [5]:
one_not_found = clip_all_df(pnnn_1, 'pnnn_1_')
print(one_not_found)

set()


In [6]:
two_not_found = clip_all_df(pnnn_2, 'pnnn_2_')
print(two_not_found)

set()


In [7]:
three_not_found = clip_all_df(pnnn_3, 'pnnn_3_')
print(three_not_found)

set()


In [8]:
four_not_found = clip_all_df(pnnn_4, 'pnnn_4_')
print(four_not_found)


set()


# Making clips that don't contain gunshots 

In [ ]:
files_korup = korup['Begin File'].unique().tolist()


In [ ]:
files_korup[0]

In [ ]:
def check_time_period_in_list (clip_start_time, start_gun, end_gun):
    # start_point_gun and end_point_gun refer to the values where gunshots occur
    # start_time refers to 
    length_clip = 10 #seconds 
    clip_end_time = clip_start_time + length_clip
    # 1. check if the gunshot is in between the start and end point considered for the clip that we want to make 
    if (( clip_start_time < start_gun < clip_end_time) or (clip_start_time < end_gun < clip_end_time)): #within 
        print('gunshot detected')
        return True
    # check if the gunshot started before the clip we want to make and it will carry on in the clip we want to make
    elif ((start_gun < clip_start_time) and (clip_start_time < end_gun < clip_end_time)): 
        print('gunshot detected')
        return True
    elif ((start_gun < clip_start_time) and (end_gun > clip_end_time)): 
        print('gunshot detected')
        return True
    else: 
        print('no gunshot  ', clip_start_time, '  ',start_gun, ' ', end_gun )
        return False 


In [ ]:
def create_clip(file, start, counter): 
    file_export_external = '/Volumes/Meli_disk/thesis_data/test/Clips_ELP/no_gunshot/'+ str(counter) + file 
    file_location = '/Volumes/Meli_Disk/thesis_data/Sounds_ELP/' + file
    second_to_milliseconds = 1000
    length_clip = 10
    audio = AudioSegment.from_wav(file_location)
    clip = audio[start*second_to_milliseconds: start*second_to_milliseconds + length_clip*second_to_milliseconds]
    clip.export(file_export_external, format='wav')



In [ ]:
# for each file, make 10 seconds long clips. 
def create_clips_without_gunshots (files_in_df, df): 
    for file in files_in_df:
        # obtain a new df that has information about gunshots within the same file 
        new_df = df[df['Begin File'].str.contains(file)]
        # obtain the begin and end time of gunshots in the file 
        time_begining_seconds = new_df['File Offset (s)'].tolist()
        time_end_seconds = (new_df['File Offset (s)'] + new_df['delta time (s)']).tolist()
        
        # ensure that the order within the times is from lowest to highest 
        sorted_pairs = sorted(zip(time_begining_seconds, time_end_seconds))
        sorted_begin_times, sorted_end_times = zip(*sorted_pairs)
        
        # transform them back in lists 
        sorted_begin_times = list(sorted_begin_times)
        sorted_end_times = list(sorted_end_times)

        # Get the file 
        file_location = '/Volumes/Meli_disk/thesis_data/Sounds_ELP/' + file

        wav_file = wave.open(file_location, 'r')
        frames = wav_file.getnframes()
        rate = wav_file.getframerate()
        duration_seconds = frames / float(rate) 

        counter = 0 
        for i in range(0, int(duration_seconds) + 1, 10):
            result = False
            for start, end in zip(sorted_begin_times, sorted_end_times): 
                result = check_time_period_in_list(i,start, end)
                if result == True: # if a gunshot occurs in that specific interval, break for lop 
                    break                                   
            if result == False: 
                create_clip(file, start, counter)
                counter += 1  




In [ ]:
create_clips_without_gunshots(files_korup[:1], korup)
